# TensorFlow Data API 를 활용한 Categorical data 다루기

지금까지의 예제는 numerical 데이터만 다루었으나

categorical 데이터를 처리하게되는 경우가 많습니다.

2개의 종류만 있는 데이터의 경우에는 0, 1로 표현하면 됩니다.

많은 종류의 데이터를 쉽게 처리하기 위해

TensorFlow 는 feature_column API 를 지원합니다.

다만 Keras와 feature_column 을 동시에 사용하기 위해서는 

또다른 API 인 Data API 를 필요로 합니다.

Data API는 DNN 모델에 입력되는 데이터 파이프라인을 쉽게 다루도록 하는 API 입니다.

메모리의 데이터와 파일로 저장된 데이터를 모두 지원하지만

간단한 메모리의 데이터를 먼저 다루도록 하겠습니다.

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


categorical 한 데이터를 다루기 위해 속도 예측기의 역할을 좀더 확장해보겠습니다.

기존에는 1개의 구간의 데이터만을 학습하였는데,

이번에는 2개의 구간의 데이터를 학습하고, 

입력으로 주어지는 구간 ID ('V_ID') 에 따라

예측을 하는 모델을 설계해보겠습니다.

이전과는 다르게, input_cols 뿐 아니라 

numerical column 은 input_cols_num,

categorical column 은 input_cols_cat 으로 따로 지정한뒤 합하겠습니다.


In [0]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
#   set LABEL first
output_cols = ['vel']
#   set traffic column
input_cols_num = ['vel_t05', 'vel_t10', 'vel_t15', 'vel_t20',
               'vel_t25', 'vel_t30', 'vel_t35',  'vel_t40']
input_cols_cat = ['V_ID']
input_cols= input_cols_num + input_cols_cat

path = './gdrive/My Drive/2020_AI_Class/03.Deep Learning/traffic_model/'

2개의 구간의 속도 데이터 'traffic_data-2link.csv' 에
앞서 다루었던 training/test 데이터 나누기,
이상치 제거, scaling을 모두 적용하겠습니다.


In [0]:
traffic_data = pd.read_csv(path+'traffic_data_2link.csv', index_col = 0)
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(traffic_data, test_size = 1024)
vel_upper_limit = train_data['vel'].quantile(q=0.98)
train_data = train_data[train_data['vel']<=vel_upper_limit]

from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(train_data['vel'].values.reshape(-1,1))

def normalize_numeric(dataframe, input_col_list, output_col_list):
    return_df = dataframe.copy()
    for col in input_col_list:
        return_df.loc[:, col] = pd.DataFrame(
            scaler.transform(return_df[col].values.reshape(-1, 1)),
            columns=[col], index=return_df.index)
    for col in output_col_list:
        col_backup = 'backup_'+col
        return_df[col_backup] = return_df[col].copy()
    for col in output_col_list:
        return_df.loc[:, col] = pd.DataFrame(
            scaler.transform(return_df[col].values.reshape(-1, 1)),
            columns=[col], index=return_df.index)
    return return_df

train_data = normalize_numeric(train_data, input_cols_num, output_cols)
test_data = normalize_numeric(test_data, input_cols_num, output_cols)


이제 데이터프레임의 데이터를 Data API 에서 지원하는 dataset 형태로 
변환하는 함수 df_to_dataset 을 만듭니다.

함수에 batch_size에 원하는 값을 넣어주어
minibatch의 크기를 지정하게됩니다.


In [0]:
def df_to_dataset(dataframe, input_col_list, output_col_list,
                  batch_size, training=True):

    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(dataframe[input_col_list]), 
         dataframe[output_col_list].values ))
    if training :
      dataset = dataset.shuffle(buffer_size= len(dataframe))
    dataset = dataset.batch(batch_size= batch_size)
    return dataset

함수가 제대로 동작하는지 확인해봅시다.


In [12]:
train_dataset = df_to_dataset(train_data, input_cols, output_cols, 4)

example_batch = next(iter(train_dataset))

print(list(example_batch[0].keys()))
print(example_batch[0]['V_ID'])
print(example_batch[1] )

['vel_t05', 'vel_t10', 'vel_t15', 'vel_t20', 'vel_t25', 'vel_t30', 'vel_t35', 'vel_t40', 'V_ID']
tf.Tensor([b'0010VS00031' b'0010VS00029' b'0010VS00029' b'0010VS00029'], shape=(4,), dtype=string)
tf.Tensor(
[[0.87945586]
 [0.75821666]
 [0.18776686]
 [0.8589018 ]], shape=(4, 1), dtype=float64)


출력 결과의 첫번째 줄은 input의 column을 의미합니다.
두번째 줄은 categorical column 인 'V_ID' 의 값을 보여줍니다.
세번째 줄은 label을 보여줍니다.

feature_column 은 모두  feature_columns 리스트에 저장하도록 합니다.

숫자형 데이터는 numeric_column 으로 변환할 수 있습니다.

In [0]:
feature_columns = []
for col in input_cols_num:
    feature_columns.append(feature_column.numeric_column(col))


Categorical column 은 두가지 과정을 거쳐야 DNN이 인식할 수 있습니다.

먼저 categorical_column_with_vocabulary_list 를 통해 category를 저장합니다.


In [0]:
V_ID_list = train_data['V_ID'].unique().tolist()
V_ID_column = feature_column.
categorical_column_with_vocabulary_list('V_ID', V_ID_list)


다음으로 indicator_column 을 통해 one hot encoding 을 하도록 한 뒤 feature_column에 저장됩니다.

In [0]:
feature_columns.append(feature_column.indicator_column(V_ID_column))


feature_column 을 통해 변환된 값을 출력하는 함수 demo를 작성하고 그 결과를 확인해봅시다.


In [0]:
def demo(feature_column_list):
  feature_layer = keras.layers.DenseFeatures(feature_column_list)
  print(feature_layer(example_batch[0]).numpy())

demo(feature_columns)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0.         1.         0.83574975 0.8021847  0.8381331  0.78699106
  0.8900695  0.8393247  0.83535254 0.92075473]
 [1.         0.         0.8764647  0.85620654 0.8797418  0.8672294
  0.87884796 0.858292   0.83515394 0.8015889 ]
 [0.         1.         0.8512413  0.83396226 0.82502484 0.8613704
  0.86305857 0.8398213  0.866137   0.8308838 ]
 [0.         1.         0.8266137  0.83763653 0.81618667 0.8182721
  0.87070507 0.84399205 0.8397219  0.8180735 ]]


Warining 은 무시하고 결과만 살펴보면 다음과 같습니다.

각 행의 앞의 0 과 1로만 구성된 두 숫자는 V_ID가 V_ID_list의 순서대로 One hot encoding 이 된 결과입니다.
0010VS_00031 은 [1, 0]
0010VS_00029 은 [0, 1] 로 표기됩니다.

뒤의 숫자들은 vel_t05 부터 vel_t40 까지의 값입니다.



이제 batch_size를 큰 값으로 늘린뒤에 DNN을 컴파일해봅시다.


In [0]:
train_dataset = df_to_dataset(train_data, input_cols, output_cols,
                              1024)

model = keras.Sequential()
model.add(keras.layers.DenseFeatures(feature_columns))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(1, activation=None))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse', metrics=['mape'])


이전에 사용한 Sequential API와 차이는 다음과 같습니다.

1. DenseFeatures 를 통해 feature_column 을 입력으로 전달함
2. 첫 레이어인 DenseFeatures 에는 input_size 가 없음.

현재 compile은 되어있으나, DenseFeatures를 사용하는 경우, 
input size를 현재 모델이 알 수가 없어 fit 전에는 summary를 출력할 수 없습니다.(가능한 방법이 있음)

이제 모델을 학습시켜 봅니다.



In [0]:
model.fit(train_dataset, epochs= 10)
model.summary()


Train for 99 steps
Epoch 1/10
99/99 [==============================] - 2s 22ms/step - loss: 0.0445 - mape: 1095.2314
Epoch 2/10
99/99 [==============================] - 1s 13ms/step - loss: 0.0017 - mape: 492.9348
Epoch 3/10
99/99 [==============================] - 1s 14ms/step - loss: 0.0016 - mape: 279.1785
Epoch 4/10
99/99 [==============================] - 1s 13ms/step - loss: 0.0015 - mape: 186.0617
Epoch 5/10
99/99 [==============================] - 1s 14ms/step - loss: 0.0015 - mape: 33.5221
Epoch 6/10
99/99 [==============================] - 1s 13ms/step - loss: 0.0015 - mape: 140.1088
Epoch 7/10
99/99 [==============================] - 1s 13ms/step - loss: 0.0015 - mape: 92.9295
Epoch 8/10
99/99 [==============================] - 1s 14ms/step - loss: 0.0014 - mape: 45.3461
Epoch 9/10
99/99 [==============================] - 1s 14ms/step - loss: 0.0014 - mape: 141.1759
Epoch 10/10
99/99 [==============================] - 1s 14ms/step - loss: 0.0014 - mape: 23.9314
Model: "seque

마찬가지로 퍼센트 에러의 계산과 test data 에 대한 evaluation 결과를 확인하겠습니다.


In [0]:
train_predict= model.predict(train_dataset)

train_predict =pd.DataFrame(scaler.inverse_transform(train_predict), index= train_data.index, columns=['prediction'])
percentage_error = (train_predict['prediction'] - train_data['backup_vel']).abs()/ train_data['backup_vel']*100
percentage_error

90247      0.628459
134585     0.861531
30660      0.691448
43662      2.643143
51682     11.990841
            ...    
83891      7.502252
55330      4.720992
78597      6.508076
93770      8.655577
169727    12.493389
Length: 100465, dtype: float64

In [0]:
percentage_error.mean()

15.788795751400574

In [0]:
test_dataset = df_to_dataset(test_data, input_cols, output_cols, 1024,training=False)

model.evaluate(test_dataset)


1/1 [==============================] - 0s 336ms/step - loss: 0.0014 - mape: 3.3777


[0.0013645220315083861, 3.3777223]